In [1]:
import os
if os.getcwd().endswith('nb'):
    os.chdir('..')


from lib.pdf_converter import pdf_to_txt_list

import json
import re
import requests

DATA_DIR = "data"
NAME = "cs514"

save_dir = os.path.join(DATA_DIR, NAME)
url_list_path = os.path.join(save_dir, "urls.txt")

def url_to_file_path(url):
    return os.path.join(save_dir, "pdfs", url.split('/')[-1])

def download_url(url):
    # make save directory
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    file_path = url_to_file_path(url)
    # check if file exists before downloading
    if not os.path.exists(file_path):
        with open(file_path, 'wb') as f:
            print("Downloading {}".format(url))
            f.write(requests.get(url).content)

In [2]:
# Download the PDF files
urls = []
with open(url_list_path, 'r') as f:
    for line in f:
        urls.append(line.strip())



for url in urls:
    download_url(url)

In [4]:
# Create a dictionary of PDF texts with keys as the PDF file names and page numbers with text as the values
# text_dict = {
    # page_name: {
    #     "link": "pdf_url#page=page_number",
    #     "text": "text_on_page"
    #     "embedding": [x1, x2, x3, ...]
    # }
# }
text_dict = {}

for url in urls:
    file_path = url_to_file_path(url)
    if not os.path.exists(file_path):
        download_url(url)
    print("Processing {}".format(file_path))
    pages = pdf_to_txt_list(file_path)
    file_name = file_path.split('/')[-1]
    for slide_num, page in enumerate(pages):
        slide_num += 1
        # Use regex to search for "{digits}\n\n\f"
        search_result = re.search(r'\n\n\d+\n\n', page)
        if search_result:
            # Get the page number
            page_num = int(search_result.group(0)[2:-2])
            # Remove the page number from the text
            page = page[:search_result.start()]
        else:
            print("Warning: No page number found for {} slide {}".format(file_name, slide_num))
        
        page_name = f"{file_name[:-4]}-{page_num}"
        link = f"{url}#page={slide_num}"
        
        text_dict[page_name] = {
            "link": link,
            "text": page
        }

# Save the dictionary to a json file with file name as the directory name
with open(f"data/{NAME}/texts.json", 'w') as f:
    json.dump(text_dict, f, indent=4)

Processing data/cs514/pdfs/lecture1.pdf
Processing data/cs514/pdfs/lecture2.pdf
Processing data/cs514/pdfs/lecture3.pdf
Processing data/cs514/pdfs/lecture4.pdf
Processing data/cs514/pdfs/lecture5.pdf
Processing data/cs514/pdfs/lecture6.pdf
Processing data/cs514/pdfs/lecture7.pdf
Processing data/cs514/pdfs/lecture8.pdf
Processing data/cs514/pdfs/lecture9.pdf
Processing data/cs514/pdfs/lecture10.pdf
Processing data/cs514/pdfs/lecture11.pdf
Processing data/cs514/pdfs/lecture12.pdf
Processing data/cs514/pdfs/lecture13.pdf
Processing data/cs514/pdfs/lecture14.pdf
Processing data/cs514/pdfs/lecture15.pdf
Processing data/cs514/pdfs/lecture16.pdf
Processing data/cs514/pdfs/lecture17.pdf
Processing data/cs514/pdfs/lecture18.pdf
Processing data/cs514/pdfs/lecture19.pdf
Processing data/cs514/pdfs/lecture20.pdf
Processing data/cs514/pdfs/lecture21.pdf
Processing data/cs514/pdfs/lecture22.pdf
Processing data/cs514/pdfs/lecture23.pdf
Processing data/cs514/pdfs/lecture24.pdf
